### 변수 변화

In [ ]:
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler # min-max 스케일링
from sklearn.preprocessing import StandardScaler # 표준화
from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns

# os 에 따른 Font 깨짐 제거를 위해 Font 지정
if os.name == 'nt': # Windows
    font_family = 'Malgun Gothic'
else: #Mac
    font_family = 'AppleGothic'
# 값이 깨지는 문제 해결을 위해 파라미터 값 설정
sns.set(font = font_family, rc = {"axes.unicode_minus" : False}) 

In [ ]:
train = pd.read_csv(r'C:\Users\user\Desktop\python workplace\train.csv')
test = pd.read_csv(r'C:\Users\user\Desktop\python workplace\test.csv')

train_test_data = [train,test]

In [ ]:
type(train_test_data)


In [ ]:
type(train)

In [ ]:
data = train.copy()
print(data.shape)
data.describe(include = object) # 오브젝트 타입만 보고싶을때

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
numerical_feats = data.dtypes[data.dtypes == 'int64'].index
numerical_feats = numerical_feats.append(data.dtypes[data.dtypes == 'float64'].index)
print('연속형 변수 카운트 :', len(numerical_feats))

In [ ]:
numerical_feats

In [ ]:
# skew() -> 왜도를 측정하기위한, -2 ~ +2는 치우침이 없는 데이터, -2 미만은 Negative Skew, +2 초과는 Positive Skew
# Positive Skewness는 오른쪽에 꼬리를 가진(왼쪽에 데이터가 많은)형태이고, Negative Skewness는 왼쪽에 꼬리를 가진(오른쪽에 데이터가 많은) 형태

# kurt() -> 첨도를 측정하기 위한, 확률분포의 뾰족한 정도. 표준정규분포와 비교하여 얼마나 뽀족한가 측정 지표
# kurt = 0 : 표준정규분포정도
# kurt < 0 : 표준정규분포보다 둥글다
# kurt > 0 : 표준정규분포보다 뾰족

for col in numerical_feats:
    print('{:15}'.format(col),
          'Skewness: {:05.2f}'.format(data[col].skew()),
          'Kurosis: {:06.2f}'.format(data[col].kurt())
        )
    

In [ ]:
f, ax = plt.subplots(figsize=(8,4))
sns.distplot(data['Fare'])

In [ ]:
data['Log_fare'] = np.log1p(data['Fare'])
f,ax = plt.subplots(figsize = (8,4))
sns.distplot(data['Log_fare'])

print('Skewness: {:.3f}'.format(data['Log_fare'].skew()))
print('Kurtosis: {:.3f}'.format(data['Log_fare'].kurt()))


In [ ]:
data['Fare'].sort_values()
data['Log_fare'].sort_values()

In [ ]:
fig = plt.figure(figsize=(10,5))

fig.add_subplot(1,2,1) # 행,렬, 그림 번호 -> 1개의 행에 2개의 열을 가진 첫번째 그림
res = stats.probplot(data['Fare'], plot = plt)
fig.add_subplot(1,2,2)
res = stats.probplot(data['Log_fare'],plot = plt)

In [ ]:
train_test_data[0].head()

In [ ]:
train_test_data[0]['Age'].isnull().sum()

In [ ]:
for dataset in train_test_data:
    dataset['Title'] =  dataset['Name'].str.extract('([\w]+)\.',expand=False)
    
for dataset in train_test_data:
    dataset['Title'] = dataset['Title'].apply(lambda x:0 if x == 'Mr' else 1 if x == 'Miss' else 2 if x=='Mrs' else 3 if x == 'Master' else 4)

for dataset in train_test_data:
    dataset['Age'].fillna(dataset.groupby('Title')['Age'].transform('median'),inplace=True)


In [ ]:
# Min-Max 정규화
scaler = MinMaxScaler()

for dataset in train_test_data:
    array = dataset['Age'].values.reshape(-1,1) # 2D array로 변환
    scaler.fit(array)  #스케일링에 필요한 값(최소값, range 등) 계산
    dataset['AgeScale'] = pd.Series(scaler.transform(array).reshape(-1))

train_test_data[0].head()

In [ ]:
# Min-Max 정규화 전 Age 변수의 min, max 값
print(train['Age'].min())
print(train['Age'].max())

# Min-Max 정규화 후 Agescale 변수의 min, max 값
print(train['AgeScale'].min())
print(train['AgeScale'].max())



In [ ]:
# Min-Max 정규화 전, 후 Age 속성과 AgeScale 속성의 산점도 비교
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
scatter_plot = sns.scatterplot(x = 'Age',y='Fare',hue = 'Sex',data=train)
plt.title('Age (Original)', fontsize = 16)

plt.subplot(1,2,2)
scatter_plot = sns.scatterplot(x = 'AgeScale',y='Fare',hue = 'Sex',data=train)
plt.title('Age (Scaled)', fontsize = 16)

In [ ]:
train_test_data[0].head()

In [ ]:
for dataset in train_test_data:
    dataset['Fare'].fillna(dataset.groupby('Pclass')['Fare'].transform('median'),
                           inplace = True)
# 표준화 수행
scaler = StandardScaler()

for dataset in train_test_data:
    array = dataset['Fare'].values.reshape(-1,1)
    scaler.fit(array)
    dataset['FareScale'] = pd.Series(scaler.transform(array).reshape(-1))
train_test_data[0]

In [ ]:
# 표준화 전 Fare 변수의 mean, var 값
print(train["Fare"].mean())
print(train["Fare"].var())
print("     ")
# 표준화 후 Fare 변수의 mean, var 값
print(train['FareScale'].mean())
print(train['FareScale'].var())


In [ ]:
# 표준화 전, 후 Fare 변수와 FareScale 변수의 산점도 비교

plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
scatter_plot = sns.scatterplot(x='Fare',y='AgeScale',hue = 'Sex', data=train)
plt.title("Fare (Original)", fontsize=16)

plt.subplot(1,2,2)
scatter_plot = sns.scatterplot(x='FareScale',y='AgeScale',hue = 'Sex', data=train)
plt.title("Fare (Scaled)", fontsize=16)

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
dis_plot = sns.distplot(train['Fare'], label = "Fare")
plt.title('Fare (Origianl)', fontsize=16)

plt.subplot(1,2,2)
dis_plot = sns.distplot(train['FareScale'], label = "FareScale")
plt.title('Fare (Scaled)', fontsize=16)
plt.show()